In [1]:
using LinearAlgebra#hide
BLAS.set_num_threads(1)#hide
using DFWannier
assets_dir = joinpath(splitdir(pathof(DFWannier))[1], "../test/assets")

"/home/runner/work/DFWannier.jl/DFWannier.jl/src/../test/assets"

We first read the colinear Hamiltonian from the outputs of wannier90.

In [2]:
hami = read_hamiltonian(joinpath(assets_dir, "wanup.chk"),
                        joinpath(assets_dir, "wandn.chk"),
                        joinpath(assets_dir, "wanup.eig"),
                        joinpath(assets_dir, "wandn.eig"))

271-element Vector{DFWannier.TBBlock{Float64, Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}, DFWannier.ColinMatrix{ComplexF64, Matrix{ComplexF64}}}}:
 DFWannier.TBBlock{Float64, Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}, DFWannier.ColinMatrix{ComplexF64, Matrix{ComplexF64}}}([-4, 1, 1], Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[-12.531005074417266 Å, -2.0885008457362106 Å, -2.0885008457362106 Å], ComplexF64[-0.001320111518912962 - 0.0006602874547082939im 0.0 + 0.0im … 0.0010000203899663078 + 1.5089681278375474e-5im 0.0 + 0.0im; -0.00018862557944137378 + 6.094661959995741e-5im 1.9711880509858146e-6 - 9.857447186097563e-8im … -1.4546615393249122e-5 + 6.938132318480357e-6im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -8.392033287142908e-5 - 5.583918090797921e-6im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … -8.514935181589618e-5 - 4.6433696217569325e-6im -0.0001317146776041345 - 9.472334986979389e-7im], ComplexF64[-0.00396033455

We can the generate the bandstructure by first defining a k-path and then perform the
interpolation.

In [3]:
structure = read_w90_input(joinpath(assets_dir, "wanup.win")).structure

Structure
    cell parameters:
	 a = (4.177001691472421 Å, 2.0885008457362106 Å, 2.0885008457362106 Å)
	 b = (2.0885008457362106 Å, 4.177001691472421 Å, 2.0885008457362106 Å)
	 c = (2.0885008457362106 Å, 2.0885008457362106 Å, 4.177001691472421 Å)
    nat: 4
    ntyp: 3
Atom[
Atom
    name: Ni1
    position_cryst: (0.5, 0.5, 0.5)
    element: symbol: Ni
Z: 28
name: Nickel
atomic_weight: 58.6934
color: (0.49744411383230336, 0.0019073777370870527, 0.0)

    projections: Projection[Orbital: d
start index: 6
last index: 10
]
    magnetization: [0.0, 0.0, 0.0]
    dftu:
DFTU
  l: Int64 -1
  U: Float64 0.0
  J0: Float64 0.0
  α: Float64 0.0
  β: Float64 0.0
  J: Array{Float64}((1,)) [0.0]

, 
Atom
    name: Ni
    position_cryst: (0.0, 0.0, 0.0)
    element: symbol: Ni
Z: 28
name: Nickel
atomic_weight: 58.6934
color: (0.49744411383230336, 0.0019073777370870527, 0.0)

    projections: Projection[Orbital: d
start index: 1
last index: 5
]
    magnetization: [0.0, 0.0, 0.0]
    dftu:
DFTU
  l: In

First we create some high symmetry kpoints
then we explicitely interpolate between the high symmetry kpoints to form
`bands_kpoints`.

In [4]:
kpoints = [Vec3(0.0, 0.0, 0.5),
           Vec3(0.0, 0.5, 0.5),
           Vec3(0.5, 0.5, 0.5),
           Vec3(0.5, 0.5, 0.0),
           Vec3(0.5, 0.0, 0.0),
           Vec3(0.0, 0.0, 0.0)]
band_kpoints = eltype(kpoints)[]
for i = 1:length(kpoints)-1
    for α in range(0, 1, 20)
        push!(band_kpoints, Vec3((1-α) .* kpoints[i] .+ α .* kpoints[i+1]))
    end
end

In order to calculate the magnetic exchanges we need to specify the fermi level (e.g. can be found in an nscf output file),
and we need to specify the atoms we want to calculate the exchanges between.
We set the number of k points used for the kpoint interpolation, and number of frequency points to calculate the
contour integral (`n_ωh`, `n_ωv`).

In [5]:
exch = calc_exchanges(hami, structure[element(:Ni)], 12.0; nk=(5,5,5), n_ωh = 300, n_ωv = 30)

Calculating H(k)... 100%|████████████████████████████████| Time: 0:00:02
Calculating contour G(ω)... 100%|████████████████████████| Time: 0:00:01


4-element Vector{DFWannier.Exchange2ndOrder{Float64}}:
 atom1:name: Ni1, pos: [0.5, 0.5, 0.5]
 atom2:name: Ni1, pos: [0.5, 0.5, 0.5]
 J: -8020.974400959045
 atom1:name: Ni1, pos: [0.5, 0.5, 0.5]
 atom2:name: Ni, pos: [0.0, 0.0, 0.0]
 J: -0.03274539633413874
 atom1:name: Ni, pos: [0.0, 0.0, 0.0]
 atom2:name: Ni1, pos: [0.5, 0.5, 0.5]
 J: -0.03244107675378643
 atom1:name: Ni, pos: [0.0, 0.0, 0.0]
 atom2:name: Ni, pos: [0.0, 0.0, 0.0]
 J: -8017.172503230155

This leads to a list of exchanges where each holds the J matrix, whose trace is the actual exchange between the sites specified
by `atom1` and `atom2`.

To calculate the exchange between the atoms in the central unit cell and those in a shifted one we can use R.
In this specific case we are calculating the exchanges towards the unit cell shifted twice along the `b` cell vector.

In [6]:
exch = calc_exchanges(hami, structure[element(:Ni)], 12.0, R=(0,2,0); nk=(5,5,5), n_ωh = 300, n_ωv = 30)

Calculating contour G(ω)... 100%|████████████████████████| Time: 0:00:01


4-element Vector{DFWannier.Exchange2ndOrder{Float64}}:
 atom1:name: Ni1, pos: [0.5, 0.5, 0.5]
 atom2:name: Ni1, pos: [0.5, 0.5, 0.5]
 J: -0.0011082790509930043
 atom1:name: Ni1, pos: [0.5, 0.5, 0.5]
 atom2:name: Ni, pos: [0.0, 0.0, 0.0]
 J: 0.003374844058546161
 atom1:name: Ni, pos: [0.0, 0.0, 0.0]
 atom2:name: Ni1, pos: [0.5, 0.5, 0.5]
 J: 27.20417579157729
 atom1:name: Ni, pos: [0.0, 0.0, 0.0]
 atom2:name: Ni, pos: [0.0, 0.0, 0.0]
 J: -0.001563985572899999